In [1]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import time
import re
import numpy as np
pd.options.display.max_rows=1000

Next, use the two log files to determine time intervals (hours) when the scheduler was unresponsive. Do this by looking for records that are the "sbatch" command from user 9204 that have return code 1 and an execution time of ~20 seconds (at least more than 15). These are commands where the scheduler timed out in responding.

3. Calculate some descriptive statistics about how often the scheduler was unresponsive, how long these periods of time were, and create a time series plot of when the scheduler was having difficulties.

4. Finally, combine the time series information from the two datasets together to see how well correlated heavy job-completion load is with the unresponsiveness, and to see if there is some threshold of job completions per hour that generally results in unresponsiveness.

In [2]:
slurm_wrapper_ce5 = pd.read_csv('slurm_wrapper_ce5.log',sep = ' - ', engine = 'python',header=None)
slurm_wrapper_ce6 = pd.read_csv('slurm_wrapper_ce6.log',sep = ' - ', engine = 'python',header=None)

In [3]:
slurm_wrapper_ce5.columns = ['DATE','USER', 'RETRY', 'TIMELAPSE', 'RETURNCODE', 'JOBID']
slurm_wrapper_ce6.columns = ['DATE','USER', 'RETRY', 'TIMELAPSE', 'RETURNCODE', 'JOBID']

In [4]:
slurm_wrapper_ce5.insert(0, 'SERVERID', 'ce5')
slurm_wrapper_ce6.insert(0, 'SERVERID', 'ce6')

In [5]:
slurm_wrapper_ce5['DATE'] = pd.to_datetime(slurm_wrapper_ce5['DATE'])
slurm_wrapper_ce6['DATE'] = pd.to_datetime(slurm_wrapper_ce6['DATE'])

In [6]:
slurm_wrapper_ce5['TIME'] = pd.to_datetime(slurm_wrapper_ce5['DATE']).dt.strftime("%H:%M:%S:%f")
slurm_wrapper_ce6['TIME'] = pd.to_datetime(slurm_wrapper_ce6['DATE']).dt.strftime("%H:%M:%S:%f")

In [7]:
slurm_wrapper_ce5['DATE'] = pd.to_datetime(slurm_wrapper_ce5['DATE']).dt.date
slurm_wrapper_ce6['DATE'] = pd.to_datetime(slurm_wrapper_ce6['DATE']).dt.date

In [9]:
slurm_append = slurm_wrapper_ce5.append(slurm_wrapper_ce6)

In [16]:
slurm_append['TIMELAPSE'] = slurm_append['TIMELAPSE'].str.replace(r'[a-z]', ' ')

C:\Users\j0909\AppData\Local\Temp/ipykernel_5556/660193185.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  slurm_append['TIMELAPSE'] = slurm_append['TIMELAPSE'].str.replace(r'[a-z]', ' ')


In [11]:
pd.options.display.max_colwidth=10000000
slurm_append.dtypes

SERVERID      object
DATE          object
USER          object
RETRY         object
TIMELAPSE     object
RETURNCODE    object
JOBID         object
TIME          object
dtype: object

In [13]:
slurm_append = slurm_append[['SERVERID','DATE','TIME','USER','RETRY','TIMELAPSE','RETURNCODE','JOBID']]

In [63]:
slurm_append.to_pickle('slurm_append')

In [17]:
slurm_append

,SERVERID,DATE,TIME,USER,RETRY,TIMELAPSE,RETURNCODE,JOBID
0,ce5,2020-10-16,08:15:39:278699,user 0,retry 0,0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']"
1,ce5,2020-10-16,08:18:08:313309,user 0,retry 0,0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']"
2,ce5,2020-10-16,08:22:48:128689,user 0,retry 0,0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']"
3,ce5,2020-10-16,08:25:13:257408,user 0,retry 0,0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-10']"
4,ce5,2020-10-16,08:31:01:460723,user 0,retry 0,0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '-S', '2020-10-14']"
...,...,...,...,...,...,...,...,...
4776515,ce6,2021-10-07,21:59:35:014602,user 9221,retry 0,0.060086965560913086,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u', 'lscpilot']"
4776516,ce6,2021-10-07,21:59:35:238970,user 9202,retry 0,0.09804415702819824,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u', 'cmslocal']"
4776517,ce6,2021-10-07,21:59:57:265189,user 9203,retry 0,0.02454972267150879,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u', 'uscmslocal']"
4776518,ce6,2021-10-07,22:00:04:024360,user 9201,retry 0,0.03941917419433594,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u', 'cmspilot']"


In [18]:
slurm_append['TIMELAPSE'] = pd.to_numeric(slurm_append['TIMELAPSE'])

In [19]:
slurm_9204_timeout = slurm_append.loc[(slurm_append['USER']  == 'user 9204') & (slurm_append['TIMELAPSE'] > 15)]

In [22]:
TEST = slurm_9204_timeout[slurm_9204_timeout['JOBID'].str.contains('sbatch')]

In [26]:
TEST.to_pickle('TEST')

In [27]:
pd.read_pickle('TEST')

,SERVERID,DATE,TIME,USER,RETRY,TIMELAPSE,RETURNCODE,JOBID
49958,ce5,2020-10-18,06:53:44:272915,user 9204,retry 0,20.038464,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55c1dd302460.1559883/bl_B3aCvf']"
49972,ce5,2020-10-18,06:54:04:322412,user 9204,retry 1,20.048906,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55c1dd302460.1559883/bl_B3aCvf']"
50467,ce5,2020-10-18,07:47:25:825172,user 9204,retry 0,20.082628,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55c1d9e86b90.1559883/bl_fa5Tsv']"
50473,ce5,2020-10-18,07:47:45:871008,user 9204,retry 1,20.045221,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55c1d9e86b90.1559883/bl_fa5Tsv']"
50582,ce5,2020-10-18,07:53:33:972840,user 9204,retry 0,20.041486,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55c1dcd1d3c0.1559883/bl_x3mVd1']"
...,...,...,...,...,...,...,...,...
4662070,ce6,2021-09-24,12:56:56:057323,user 9204,retry 0,19.568814,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55b43cfb0b60.2522340/bl_qrRDcs']"
4662752,ce6,2021-09-24,13:29:48:498748,user 9204,retry 0,20.085085,returncode 1,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55b438a30440.2522340/bl_JqREcD']"
4667202,ce6,2021-09-24,20:59:45:540176,user 9204,retry 0,16.153547,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55b43cc491b0.2522340/bl_PQMqxI']"
4737128,ce6,2021-10-02,19:03:06:524282,user 9204,retry 0,15.063486,returncode 0,"command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x55a9a0918330.3718051/bl_IRg3cU']"
